Лабораторная работа 1. Задание 2


In [1]:
import pandas as pd


In [2]:
url = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%90%D0%9A%D0%A4_%D0%91_%D0%9D_000006970.html"

try:
    tables = pd.read_html(url, skiprows=11)
except Exception as e:
    print(f"Ошибка при загрузке: {e}")


In [3]:
# Обрабатываем загруженные данные
if 'tables' in locals() and len(tables) > 0:
    # Берем первую таблицу
    df = tables[0]
    
    # Устанавливаем строку 1 как заголовки
    df.columns = df.iloc[1]
    df = df.drop(df.index[0:2]).reset_index(drop=True)
    
    # Убираем последние 2 столбца (о целевом обучении)
    df = df.iloc[:, :-2]
    
    # Убираем столбцы о целевом обучении (если они еще есть)
    columns_to_remove = ['Идентификационный номер заказчика целевого обучения (для целевого приема)', 
                        'Номер предложения (для целевого приема)']
    df = df.drop(columns=columns_to_remove, errors='ignore')
    
    # Настройки для красивого вывода
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 15)
    pd.set_option('display.max_rows', None)
    
else:
    print("Не удалось загрузить таблицы")


In [4]:
# Приводим количественные переменные к числовому типу
numeric_columns = ['Сумма баллов', 'АиНМА / Математика', 'ПФ / Информатика и ИКТ / Физика / Химия', 
                   'Русский язык', 'Сумма баллов за ИД']

for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [5]:
# Вычисляем необходимые значения
total_students = len(df)
mean_total_score = df['Сумма баллов'].mean()
min_total_score = df['Сумма баллов'].min()
max_total_score = df['Сумма баллов'].max()

# Определяем предмет с высшим средним баллом
math_mean = df['АиНМА / Математика'].mean()
it_mean = df['ПФ / Информатика и ИКТ / Физика / Химия'].mean()
russian_mean = df['Русский язык'].mean()

if math_mean > it_mean and math_mean > russian_mean:
    best_subject = "Математика"
elif it_mean > russian_mean:
    best_subject = "ПФ/Информатика/Физика/Химия"
else:
    best_subject = "Русский язык"

# Подсчитываем количество студентов каждого типа
condition_type1 = (
    (df['Русский язык'] < russian_mean) &
    (df['АиНМА / Математика'] > math_mean) &
    (df['ПФ / Информатика и ИКТ / Физика / Химия'] > it_mean)
)

condition_type2 = (
    (df['Русский язык'] > russian_mean) &
    (df['АиНМА / Математика'] < math_mean) &
    (df['ПФ / Информатика и ИКТ / Физика / Химия'] < it_mean)
)

type1_count = condition_type1.sum()
type2_count = condition_type2.sum()

# Создаем итоговый DataFrame
summary_df = pd.DataFrame({
    "Подразделение": ["Архитектурно-строительный факультет"],
    "Уровень подготовки": ["Бакалавриат"],
    "Направление подготовки/специальность": ["Архитектура"],
    "Год": [2025],
    "Количество мест": [len(df)],
    "Предметы ЕГЭ": ["Математика, ПФ/Информатика/Физика/Химия, Русский язык"],
    "Средняя сумма баллов": [round(mean_total_score, 2)],
    "Min сумма баллов": [min_total_score],
    "Max сумма баллов": [max_total_score],
    "Предмет с высшим средним": [best_subject],
    "Кол-во студентов тип_1": [type1_count],
    "Кол-во студентов тип_2": [type2_count]
})

display(summary_df)


,Подразделение,Уровень подготовки,Направление подготовки/специальность,Год,Количество мест,Предметы ЕГЭ,Средняя сумма баллов,Min сумма баллов,Max сумма баллов,Предмет с высшим средним,Кол-во студентов тип_1,Кол-во студентов тип_2
0,Архитектурн...,Бакалавриат,Архитектура,2025,17,"Математика,...",188.76,166,235,Математика,0,2
